In [1]:
import numpy as np
import pandas as pd
import json
import os

In [14]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQE\Differential Evolution\Noise"
cutoffs = [2, 4, 8, 16, 32, 64, 128, 256]
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [15]:
folders

['AHO', 'DW', 'QHO']

In [24]:
converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
parallel_run_time = []
total_VQE_time = []
total_device_time = []
potential= []
cutoffss =[]
percentages= []

for f in folders:

    for cutoff in cutoffs:

        data_path = os.path.join(folder_path, f, "{}_{}.json".format(f, cutoff))
        
        with open(data_path, 'r') as file:
            data = json.load(file)

        potential.append(f)
        cutoffss.append(cutoff)

        exact_min_e = np.min(data['exact_eigenvalues'])
        exact_e.append(exact_min_e)
        
        converged_runs.append(sum(data['success']))
        mean_iters.append(int(np.round(np.mean(data['num_iters']))))

        min_e.append(np.min(data['results']))
        delta_min_e.append(abs(exact_min_e - np.min(data['results'])))

        median_e.append(np.median(data['results']))   
        delta_median_e.append(abs(exact_min_e - np.median(data['results']))) 

        total_VQE_time.append(data['total_VQE_time'])
        parallel_run_time.append(data['parallel_run_time'])
        total_device_time.append(data['total_device_time'])

        dev_perc = (pd.Timedelta(data['total_device_time']).total_seconds() / pd.Timedelta(data['total_VQE_time']).total_seconds()) * 100
        percentages.append(dev_perc)

    

In [26]:
df = pd.DataFrame({
    'Potential': potential,
    'cutoff': cutoffss,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    'Mean Iter': mean_iters,
    'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'total_VQE_time': total_VQE_time,
    'parallel_run_time': parallel_run_time,
    'total_device_time': total_device_time,
    'percentage_device_time': percentages
})

In [27]:
df

,Potential,cutoff,Converged Runs,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,total_VQE_time,parallel_run_time,total_device_time,percentage_device_time
0,AHO,2,100/100,71,-4.37500e-01,0.00000e+00,-4.37500e-01,0.00000e+00,-4.37500e-01,0:10:16.968768,0:00:20.682285,0:00:19.444167,3.151564
1,AHO,4,100/100,35,-1.64785e-01,4.99600e-16,-1.64785e-01,4.99600e-16,-1.64785e-01,0:06:00.354135,0:00:12.546418,0:00:11.339130,3.146663
2,AHO,8,100/100,58,3.20101e-02,1.26218e-14,3.20101e-02,1.26218e-14,3.20101e-02,0:09:47.500391,0:00:19.155892,0:00:17.377715,2.957907
3,AHO,16,100/100,291,-1.16698e-03,4.05626e-13,-1.16698e-03,4.05626e-13,-1.16698e-03,0:46:02.058423,0:02:11.105607,0:01:18.062679,2.826250
4,AHO,32,100/100,201,6.18224e-06,3.17674e-12,6.18224e-06,3.17674e-12,6.18224e-06,0:42:21.767190,0:01:20.765366,0:01:02.076913,2.442274
5,AHO,64,100/100,210,-1.94478e-08,2.68587e-11,-1.94478e-08,2.68587e-11,-1.94209e-08,1:19:02.852391,0:03:11.592004,0:01:16.349043,1.609771
6,AHO,128,100/100,201,1.40750e-12,2.12090e-11,1.40750e-12,2.12090e-11,-1.98015e-11,3:58:43.837566,0:08:27.401224,0:01:28.958301,0.621051
7,AHO,256,100/100,197,2.15004e-11,1.28240e-10,2.15004e-11,1.28240e-10,1.49740e-10,19:21:39.424037,0:37:30.189291,0:01:45.293553,0.151068
8,DW,2,100/100,37,3.57233e-01,5.55112e-17,3.57233e-01,5.55112e-17,3.57233e-01,0:07:03.767080,0:00:13.963019,0:00:11.940206,2.817634
9,DW,4,100/100,85,9.06560e-01,1.77636e-15,9.09281e-01,2.72159e-03,9.06560e-01,0:18:30.289249,0:00:37.152873,0:00:26.021284,2.343649
